# 0) GPU 확인

In [ ]:
import torch, platform, sys, subprocess, os, json
print("Python :", sys.version)
print("Platform:", platform.platform())
print("CUDA ok:", torch.cuda.is_available())
!nvidia-smi || true

Python : 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
CUDA ok: True
Thu Oct 23 07:14:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                  

# 1) RT-DETR v2 & requirements

In [ ]:
# RT-DETR v2 설치
!git clone https://github.com/pseudo-sue/AICV_pro1.git

Cloning into 'AICV_pro1'...
remote: Enumerating objects: 1245, done.
remote: Counting objects: 100% (1245/1245), done.
remote: Compressing objects: 100% (646/646), done.
remote: Total 1245 (delta 627), reused 1180 (delta 573), pack-reused 0 (from 0)
Receiving objects: 100% (1245/1245), 684.50 KiB | 1012.00 KiB/s, done.
Resolving deltas: 100% (627/627), done.


In [ ]:
# RT-DETR v2가 요구하는 라이브러리 다운
!pip -q install -r AICV_pro1/rtdetrv2_pytorch/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.6/585.6 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.5 MB/s eta 0:00:00


# 2) Kitti 데이터셋 받기

> https://www.kaggle.com/datasets/klemenko/kitti-dataset



In [ ]:
# 1. 설치
!pip install kagglehub -q

# 2. Import
import kagglehub

# 3. 다운로드 (자동 unzip 포함)

path = kagglehub.dataset_download("klemenko/kitti-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'kitti-dataset' dataset.
Path to dataset files: /kaggle/input/kitti-dataset


# 3) 전처리

## 환경 설정

In [ ]:
# =========================
# 0) 경로/설정 (YOLO 전부 제거)
# =========================
import os
import json
import shutil
import random
from PIL import Image
from sklearn.model_selection import train_test_split

# KITTI 라벨/이미지 경로
kitti_train_label_dir = os.path.join(path, "data_object_label_2/training/label_2")
kitti_test_label_dir  = os.path.join(path, "data_object_label_2/testing/label_2")
train_image_dir       = os.path.join(path, "data_object_image_2/training/image_2")
test_image_dir        = os.path.join(path, "data_object_image_2/testing/image_2")

# DETR용 COCO 전체 JSON 경로
output_train_json = "/content/kitti/labels/RE_DETR/before_split.json"

# 이미지 스플릿(폴더) — 이미지만 복사합니다. (라벨 폴더 없음)
train_img_dir = "/content/kitti/images/train"
val_img_dir   = "/content/kitti/images/val"
test_img_dir  = "/content/kitti/images/test"

# 스플릿 정보 저장
split_info_path = "/content/kitti/split_info.txt"

# 스플릿된 COCO JSON 경로
train_out_path = "/content/kitti/labels/RE_DETR/train_split.json"
val_out_path   = "/content/kitti/labels/RE_DETR/val_split.json"
test_out_path  = "/content/kitti/labels/RE_DETR/test_split.json"

# 클래스
classes = [
    "Car",
    "Van",
    "Truck",
    "Pedestrian",
    "Person_sitting",
    "Cyclist",
    "Tram",
    # "Misc",
    # "DontCare"
]

# 재현성 & 비율
SEED = 42
random.seed(SEED)
TRAIN_RATIO = 0.7
VAL_RATIO   = 0.2
TEST_RATIO  = 0.1
assert abs((TRAIN_RATIO + VAL_RATIO + TEST_RATIO) - 1.0) < 1e-6

## RT-DETR 전처리

In [ ]:
# =========================
# 1) KITTI → COCO(JSON) 변환 (RT-DETR용)
# =========================
def kitti_to_coco(kitti_label_dir, image_dir, output_json, classes):
    images, annotations = [], []
    ann_id = 1

    for img_id, file in enumerate(sorted(os.listdir(kitti_label_dir))): # 항상 같은 결과를 위해 sort
        if not file.endswith(".txt"):
            continue
        label_path = os.path.join(kitti_label_dir, file)

        # 이미지 확장자 결정(.png 우선, 없으면 .jpg/.jpeg)
        base = os.path.splitext(file)[0]
        img_file = None
        for ext in (".png", ".jpg", ".jpeg"):
            candidate = base + ext
            if os.path.exists(os.path.join(image_dir, candidate)):
                img_file = candidate
                break
        if img_file is None:
            continue

        img_path = os.path.join(image_dir, img_file)
        if not os.path.exists(img_path):
            continue

        with Image.open(img_path) as img:
            width, height = img.size

        images.append({
            "id": img_id,
            "file_name": img_file,
            "width": width,
            "height": height
        })

        # 라벨 읽기
        with open(label_path) as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 8:
                    continue
                cls = parts[0]
                if cls not in classes:
                    continue
                cls_id = classes.index(cls) + 1  # COCO는 1부터
                xmin, ymin, xmax, ymax = map(float, parts[4:8])
                w, h = xmax - xmin, ymax - ymin

                annotations.append({
                    "id": ann_id,
                    "image_id": img_id,
                    "category_id": cls_id,
                    "bbox": [xmin, ymin, w, h],
                    "area": w * h,
                    "iscrowd": 0
                })
                ann_id += 1

    coco_dict = {
        "images": images,
        "annotations": annotations,
        "categories": [{"id": i + 1, "name": c} for i, c in enumerate(classes)]
    }

    os.makedirs(os.path.dirname(output_json), exist_ok=True)
    with open(output_json, "w") as f:
        json.dump(coco_dict, f, indent=4)

    print(f"✅ COCO JSON saved at {output_json} | images={len(images)}, annotations={len(annotations)}")

# 실행
kitti_to_coco(
    kitti_label_dir=kitti_train_label_dir,
    image_dir=train_image_dir,
    output_json=output_train_json,
    classes=classes
)

✅ COCO JSON saved at /content/kitti/labels/RE_DETR/before_split.json | images=7481, annotations=39597


## train/val split(random seed 고정)

In [ ]:
# =========================
# 2) 이미지 스플릿 (이미지만 복사 / YOLO 라벨 복사 없음)
# =========================
for d in [train_img_dir, val_img_dir, test_img_dir]:
    os.makedirs(d, exist_ok=True)

# 전체 이미지 목록 (training에서만 사용)
train_imgs_all = sorted([f for f in os.listdir(train_image_dir)
                         if f.lower().endswith((".png", ".jpg", ".jpeg"))])

# TEST 분리
trainval_imgs, test_imgs = train_test_split(
    train_imgs_all,
    test_size=TEST_RATIO,
    random_state=SEED,
    shuffle=True
)

# TRAIN/VAL 분리
val_rel = VAL_RATIO / (TRAIN_RATIO + VAL_RATIO)
train_imgs, val_imgs = train_test_split(
    trainval_imgs,
    test_size=val_rel,
    random_state=SEED,
    shuffle=True
)

def copy_images(img_list, src_img_dir, dst_img_dir):
    for img_file in img_list:
        shutil.copy(os.path.join(src_img_dir, img_file),
                    os.path.join(dst_img_dir, img_file))

# 복사 (이미지들만)
copy_images(train_imgs, train_image_dir, train_img_dir)
copy_images(val_imgs,   train_image_dir, val_img_dir)
copy_images(test_imgs,  train_image_dir, test_img_dir)

# 스플릿 정보 저장(기존 포맷 유지)
with open(split_info_path, "w") as f:
    f.write("Train files:\n" + "\n".join(train_imgs) + "\n\n")
    f.write("Val files:\n"   + "\n".join(val_imgs)   + "\n\n")
    f.write("Test files:\n"  + "\n".join(test_imgs)  + "\n")

print(f"✅ Split 완료 (seed={SEED})")
print(f"Train: {len(train_imgs)} / Val: {len(val_imgs)} / Test: {len(test_imgs)}")
print(f"Split 기록 저장: {split_info_path}")

✅ Split 완료 (seed=42)
Train: 5235 / Val: 1497 / Test: 749
Split 기록 저장: /content/kitti/split_info.txt


## JSON 변환

In [ ]:
# =========================
# 3) 스플릿 정보 기반 COCO JSON 분할 저장
# =========================
# 3-1) split_info.txt 읽기
splits = {"train": [], "val": [], "test": []}
current = None
with open(split_info_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        if line.startswith("Train"):
            current = "train"; continue
        elif line.startswith("Val"):
            current = "val"; continue
        elif line.startswith("Test"):
            current = "test"; continue
        elif current:
            splits[current].append(line)

print({k: len(v) for k, v in splits.items()})

# 3-2) COCO 전체 JSON 로드
with open(output_train_json, "r") as f:
    coco = json.load(f)

images = coco["images"]
annotations = coco["annotations"]
categories = coco["categories"]

# 3-3) 이미지 파일명 → id 매핑
filename_to_id = {img["file_name"]: img["id"] for img in images}

# 3-4) split별 이미지 id 집합
split_ids = {k: set(filename_to_id[f] for f in v if f in filename_to_id) for k, v in splits.items()}

# 3-5) split별 JSON 구성 함수
def make_split_json(split_name):
    ids = split_ids[split_name]
    imgs = [img for img in images if img["id"] in ids]
    anns = [ann for ann in annotations if ann["image_id"] in ids]
    return {"images": imgs, "annotations": anns, "categories": categories}

# 3-6) 저장
for name, out_path in [("train", train_out_path),
                       ("val",   val_out_path),
                       ("test",  test_out_path)]:
    subset = make_split_json(name)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, "w") as f:
        json.dump(subset, f)
    print(f"✅ {name}_split.json 저장 완료 → {out_path} ({len(subset['images'])} images, {len(subset['annotations'])} anns)")

{'train': 5235, 'val': 1497, 'test': 749}
✅ train_split.json 저장 완료 → /content/kitti/labels/RE_DETR/train_split.json (5235 images, 27587 anns)
✅ val_split.json 저장 완료 → /content/kitti/labels/RE_DETR/val_split.json (1497 images, 7874 anns)
✅ test_split.json 저장 완료 → /content/kitti/labels/RE_DETR/test_split.json (749 images, 4136 anns)


# 4) 학습

In [ ]:
# -c에 설정파일
!cd /content/AICV_pro1/rtdetrv2_pytorch &&export PYTHONPATH=$PWD:$PYTHONPATH && \
torchrun --nproc_per_node=1 --master_port=29500 \
  tools/train.py \
  -c configs/rtdetrv2/rtdetrv2_r18vd_vim_up_50e_kitti_T4.yml \
  --use-amp \
  --seed 42

2025-10-23 07:17:40.883117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761203861.185425    2472 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761203861.264921    2472 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761203861.891191    2472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761203861.891235    2472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761203861.891240    2472 computation_placer.cc:177] computation placer alr

# 5) 추론

In [ ]:
# 프로젝트 루트로 이동 + PYTHONPATH 설정
!cd /content/AICV_pro1/rtdetrv2_pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
python tools/infer_pth_dir.py \
  -c configs/rtdetrv2/rtdetrv2_r18vd_vim_up_50e_kitti_T4.yml \
  -r output/rtdetrv2_kitti/best.pth \
  --im-dir /content/kitti/images/test \
  --out-dir /content/kitti/preds_rtdetr \
  --thr 0.4 \
  --classes kitti

2025-10-23 08:01:53.499100: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761206513.526367   13365 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761206513.535407   13365 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761206513.571031   13365 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761206513.571062   13365 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761206513.571068   13365 computation_placer.cc:177] computation placer alr

# 6) DETR 평가만

In [ ]:
!cd /content/AICV_pro1//rtdetrv2_pytorch && \
python tools/train.py \
  -c configs/rtdetrv2/rtdetrv2_r18vd_vim_up_50e_kitti_for_eval_T4.yml \
  -r output/rtdetrv2_kitti/best.pth \
  --device cuda:0 \
  --test-only \
  -u dataset.val.image_dir=/content/kitti/images/test \
     dataset.val.anno_path=/content/kitti/labels/test.json

2025-10-23 08:07:11.788098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761206831.809912   14674 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761206831.816039   14674 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761206831.832266   14674 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761206831.832298   14674 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761206831.832304   14674 computation_placer.cc:177] computation placer alr

# 7) pth 변환 및 다운

In [ ]:
from google.colab import files

files.download('/content/AICV_pro1/rtdetrv2_pytorch/output/rtdetrv2_kitti/best.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>